# Import

In [1]:
import numpy as np
import pandas as pd
import pickle
import os

## 파일 불러오기

In [16]:
# 파일 불러오기
with open("/Users/heogeonhyeock/Desktop/인빅_프로젝트/FC주문/FC_order.pickle","rb") as fr:
    Distance = pickle.load(fr).reset_index(drop=True)

# 데이터 전처리

In [5]:
# Null값 제거
Df_list = ["SHPR_ADDR_1","SHPR_ADDR_2","CNEE_ADDR_1","CNEE_ADDR_2"]
Df_list2 = ["SHPR_ADDR_2","CNEE_ADDR_2"]
Df_index = []
Distance = Distance[Df_list].dropna(axis=0)

# 5개 미만의 데이터 즉 노이즈 제거 (주소가 양식에 벗어남)
for i in Df_list2:
    for j in list(Distance[i].unique()):
        if len(Distance[Distance[i] == j]) < 5:
            Df_index.append(Distance[Distance[i] == j].index)
for i in Df_index:
    Distance = Distance.drop(index = i)

# 세로운 컬럼 송화인과 수화인 생성
Distance["송화인"] = Distance["SHPR_ADDR_1"] + " " + Distance["SHPR_ADDR_2"]
Distance["수화인"] = Distance["CNEE_ADDR_1"] + " " + Distance["CNEE_ADDR_2"]
Distance = Distance.drop(Df_list, axis = 1).reset_index(drop=False)
Distance_same = Distance[Distance["송화인"] == Distance["수화인"]] #같은지역
Distance = Distance[Distance["송화인"] != Distance["수화인"]] # 다른지역
Distance["distance"] = None #다른지역
Distance_same["distance"] = 0 #같은지역
Distance

NameError: name 'Distance' is not defined

# geocoder를 이용한 거리계산

In [4]:
"""
#구글 api 이용한 주소 dict 생성
import geocoder

A = list(Distance["송화인"].unique())
B = list(Distance["수화인"].unique())
C = {}

for i in A:
    for j in B:
        try:
            C[i,j] = geocoder.distance(geocoder.location(i),
                                       geocoder.location(j))
            print(i)
        except:
            print("{}번째 인덱스에서 오류".format(i))
with open("Distance_dict.pickle","wb") as fw:
    pickle.dump(C, fw)
"""

'\n#구글 api 이용한 주소 dict 생성\nimport geocoder\n\nA = list(Distance["송화인"].unique())\nB = list(Distance["수화인"].unique())\nC = {}\n\nfor i in A:\n    for j in B:\n        try:\n            C[i,j] = geocoder.distance(geocoder.location(i),\n                                       geocoder.location(j))\n            print(i)\n        except:\n            print("{}번째 인덱스에서 오류".format(i))\nwith open("Distance_dict.pickle","wb") as fw:\n    pickle.dump(C, fw)\n'

## 저장된 계산거리 불러오기

In [ ]:
with open("/Users/heogeonhyeock/Desktop/인빅_프로젝트/Distance_dict.pickle","rb") as fr:
    C = pickle.load(fr)
# 계산실패한 값 None으로 변경
distance_0 = [i for i,j in C.items() if j == 0]
for i in distance_0:
    C[i] = None

## distance 열 생성

In [5]:
for i in list(Distance.index):
    try:
        Distance["distance"][i] = C[(Distance["송화인"][i], Distance["수화인"][i])]
    except:
        print("{}번째 인덱스에서 오류".format(i))

Distance = pd.concat([Distance,Distance_same]).sort_values("index")
Distance_DF = Distance.set_index("index")
Distance_DF

<ipython-input-5-16aa811b1e82>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Distance["distance"][i] = C[(Distance["송화인"][i], Distance["수화인"][i])]


,index,송화인,수화인,distance
0,1,경기도 광주시,경상남도 김해시,284.947
1,2,경기도 광주시,경상남도 거제시,308.782
2,3,경기도 광주시,서울특별시 서초구,20.5919
3,4,경기도 광주시,경기도 김포시,51.8882
4,5,경기도 광주시,경기도 김포시,51.8882
...,...,...,...,...
1641060,1786347,경기도 광주시,경기도 수원시,27.2098
1641061,1786348,경기도 광주시,제주특별자치도 제주시,442.792
1641062,1786349,경기도 광주시,대전광역시 유성구,118.972
1641063,1786353,경기도 용인시,경상남도 하동군,247.076


## 기존데이터 불러오기

In [ ]:
with open("Distance_DF.pickle","wb") as fw:
    pickle.dump(Distance, fw)
Df_index = []
Distance = Distance.dropna(axis=0)

# 5개 미만의 데이터 즉 노이즈 제거 (주소가 양식에 벗어남)
for i in Df_list2:
    for j in list(Distance[i].unique()):
        if len(Distance[Distance[i] == j]) < 5:
            Df_index.append(Distance[Distance[i] == j].index)
for i in Df_index:
    Distance = Distance.drop(index = i)
Distance["distance"] == None
Distance

## 데이터결합 및 저장

In [17]:
Distance["distance"] = Distance_DF["distance"]
Distance
with open("FC_order_distance.pickle","wb") as fw:
    pickle.dump(Distance, fw)

# 번외

## 네이버 api이용한 거리계산

In [ ]:
"""
#네이버 api 이용한 주소 dict 생성
import requests
import asyncio
from haversine import haversine

# pip install haversine requests

NAVER_API = 
NAVER_API_ID = 
NAVER_API_SECRET = 


def fetcher(session, url, headers):
    with session.get(url, headers=headers) as response:
        if response.ok:
            result = response.json()
            try:
                address = result['addresses']
                if not address:
                    return
                x = float(address[0]['x'])
                y = float(address[0]['y'])
                return (x, y)
            except KeyError:
                return
        else:
            return


def main(fucking_address):
    headers = {
        "X-NCP-APIGW-API-KEY-ID": NAVER_API_ID,
        "X-NCP-APIGW-API-KEY": NAVER_API_SECRET
    }
    with requests.Session() as session:
        return fetcher(session, f'{NAVER_API}?query={fucking_address}', headers)


def tony_sang_seok(fucking_address):
    return main(fucking_address)


def fucking_distance(address1, address2):
    # 단위는 키로미터
    if address1 and address2:
        return haversine(address1, address2)
    else:
        return

#거리 구하기


Distance["distance"] = None
Distance_same["distance"] = 0
for i in range(len(Distance)):
    Distance["distance"][i] = fucking_distance(tony_sang_seok(Distance["송화인"][i]),
                                               tony_sang_seok(Distance["수화인"][i]))
Distance = pd.concat([Distance,Distance_same]).sort_values("index")
Distance


A = list(Distance["송화인"].unique())
B = list(Distance["수화인"].unique())
C = {}
for i in A:
    for j in B:
        C[i,j] = fucking_distance(tony_sang_seok(i),tony_sang_seok(j))
"""

# 저장된 New FC_order 불러오기

In [2]:
# Na값 제거, 주소지 중 빈도수가 5 미만인 행 삭제
with open("/Users/heogeonhyeock/Desktop/인빅_프로젝트/FC주문/FC_order_distance.pickle","rb") as fr:
    FC_order = pickle.load(fr).reset_index(drop=True)

In [3]:
FC_order

,CORP_ID,REF_ORD_NO,BKG_NO,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_CD,...,INS_ID,INS_DATE,INS_TIME,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2,distance
0,KX007,2021030365922951,KX007210303371080,7,20210303,35521,90001441,20900.0,1,4610261043_2,...,iNEXBatch,20210303,131425,50.0,N,경기도,광주시,경상남도,김해시,284.947
1,KX007,2021030365924361,KX007210303371081,7,20210303,35513,90001441,37900.0,1,4936599538_1,...,iNEXBatch,20210303,131425,53.0,N,경기도,광주시,경상남도,거제시,308.782
2,KX007,2021030365928921,KX007210303371205,7,20210303,35608,90001542,69800.0,1,3858391637_1,...,iNEXBatch,20210303,131425,6.0,N,경기도,광주시,서울특별시,서초구,20.5919
3,KX007,2021030365930451,KX007210303370967,7,20210303,35551,90001341,73700.0,1,4631307460,...,iNEXBatch,20210303,131425,10.0,N,경기도,광주시,경기도,김포시,51.8882
4,KX007,2021030365930451,KX007210303370967,7,20210303,35551,90001341,73700.0,2,2475138283,...,iNEXBatch,20210303,131425,10.0,N,경기도,광주시,경기도,김포시,51.8882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585442,KX007,2021063075245201,KX007210702671565,7,20210630,122001,90001622,125800.0,1,5519470982_18,...,iNEXBatch,20210702,101425,7.0,N,경기도,광주시,서울특별시,동작구,29.3495
1585443,KX007,2021062954478421,KX007210702674788,7,20210629,122812,90001443,16900.0,1,5002901762_12,...,iNEXBatch,20210702,122203,16.0,N,경기도,광주시,경기도,수원시,27.2098
1585444,KX007,2021063067849541,KX007210703706457,7,20210630,256,90001662,138000.0,1,5410093340,...,iNEXBatch,20210703,220152,63.0,N,경기도,광주시,제주특별자치도,제주시,442.792
1585445,KX007,2021063078029521_1,KX007210704717899,7,20210630,143118,90001443,13900.0,1,5582399098,...,BATCH,20210704,55047,34.0,N,경기도,광주시,대전광역시,유성구,118.972
